In [48]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.document_loaders import TextLoader

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from dotenv import load_dotenv
import os
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor
from langchain import hub


In [ ]:
load_dotenv(override=True)
file ="BusinessContext.txt"
API_KEY=os.getenv("OPEN_API_KEY")



documents = TextLoader(file).load()


text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(documents)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings(openai_api_key=API_KEY))
retriever = vectorstore.as_retriever()
retriever

In [58]:
tavily_search = TavilySearchResults(
    max_results=5,
    search_depth="advanced",
)

retriever_tool = create_retriever_tool(
    retriever,
    'text_retreiver',
    "Give answer based on the file",
)
# prompt = hub.pull("bhargavnimalkar/rag2")
prompt = hub.pull("hwchase17/openai-functions-agent")

llm = ChatOpenAI(
model="gpt-3.5-turbo",
temperature=0.5,
api_key=API_KEY
)


In [59]:

tools = [retriever_tool, tavily_search]
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [1]:
result = agent_executor.invoke({"input": 'who is messi?'}) 
result['output'] 

NameError: name 'agent_executor' is not defined

In [67]:
search = TavilySearchResults()
search.invoke("what is the best manga")[4]['content']

"Browse the highest-ranked manga on MyAnimeList, the internet's largest manga database. Find the top manga, novels, one-shots and more!"